In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("E:/github_repo/4DSTEM_processing/")
from FDSTEM_process import *
import tifffile
import tkinter.filedialog as tkf
plt.rcParams['font.family'] = 'Cambria'

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

In [ ]:
scan_per_pixel = 0.11272 # Å
mrad_per_pixel = 0.40877 # mrad
fd = FourDSTEM_process(raw_adr, scan_per_pixel, mrad_per_pixel)

In [ ]:
fd.spike_remove(percent_thresh=0.01, mode="lower", apply_remove=True)

In [ ]:
c_pos = fd.find_center(cbox_edge=64)
print(c_pos)

In [ ]:
fd.disk_extract(buffer_size=1)

In [ ]:
semiangle = fd.least_R*mrad_per_pixel
BF_det = np.array([0, semiangle])
ADF_det = np.array([semiangle, semiangle*2])

print(BF_det, ADF_det)

fd.virtual_stem(BF_det, ADF_det)

In [ ]:
fd.DPC(correct_rotation=True, n_theta=100, hpass=0.05, lpass=0.05)

In [ ]:
tifffile.imwrite("vHAADF.tif", self.ADF_stem)
tifffile.imwrite("dDPC.tif", self.charge_density)
tifffile.imwrite("iDPC.tif", self.potential)

In [ ]:
alpha, beta = np.cos(fd.c_theta*180/np.pi), np.sin(fd.c_theta*180/np.pi)
M = np.array([[alpha, beta, (1-alpha)*fd.c_ct[1]-beta*fd.c_ct[0]], 
              [-beta, alpha, beta*fd.c_ct[1]+(1-alpha)*fd.c_ct[0]]])

rotated_pacbed = rotation(fd.c_pacbed, M)
ri = radial_indices(fd.c_pacbed.shape, [0, fd.cropped_size], 1, center=fd.c_ct)
mirror_plane = np.zeros(rotated_pacbed.shape)
mirror_plane[:, int(fd.c_ct[1])] = 1
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(np.multiply(fd.c_pacbed, ri), cmap="jet")
ax[0].imshow(mirror_plane, cmap="gray", alpha=0.2)
ax[0].scatter(fd.c_ct[1], fd.c_ct[0], s=15, c="k")
ax[0].axis("off")
ax[1].imshow(np.multiply(rotated_pacbed, ri), cmap="jet")
ax[1].imshow(mirror_plane, cmap="gray", alpha=0.2)
ax[1].scatter(fd.c_ct[1], fd.c_ct[0], s=15, c="k")
ax[1].axis("off")
fig.tight_layout()
plt.show()

In [ ]:
start = time.process_time()
rot_angle = 3
fd.symmetry_evaluation(rot_angle, also_mirror=True)
print("%d seconds have passed"%(time.process_time()-start))

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0][0].imshow(fd.c_stack[0, 0], cmap="jet")
ax[0][0].scatter(fd.c_ct[1], fd.c_ct[0], s=15, c="k")
ax[0][0].set_title("original CBED")
ax[0][0].axis("off")
ax[0][1].imshow(rotated_stack[0, 0], cmap="jet")
ax[0][1].scatter(fd.c_ct[1], fd.c_ct[0], s=15, c="k")
ax[0][1].set_title("rotated CBED (angle=%d)"%rot_angle)
ax[0][1].axis("off")
ax[1][0].imshow(fd.c_stack[5, 5], cmap="jet")
ax[1][0].scatter(fd.c_ct[1], test.c_ct[0], s=15, c="k")
ax[1][0].set_title("another original CBED")
ax[1][0].axis("off")
ax[1][1].imshow(rotated_stack[5, 5], cmap="jet")
ax[1][1].scatter(fd.c_ct[1], fd.c_ct[0], s=15, c="k")
ax[1][1].set_title("rotated CBED (angle=%d)"%rot_angle)
ax[1][1].axis("off")
fig.tight_layout()
plt.show()